In [1]:
from tkinter import *
from tkinter import ttk
import nbimporter
import  math
import threading
import time
import sys
if('..' not in sys.path):
    sys.path.insert(0,'..')
    
from Maze.Maze import Maze
from Maths.Cord import Cord

class Viewer:
    agentsFinished=0
    
    def __init__(self, m, ip, hd, im, hdm, q,hdq, c ):#,m, a, h,t):
        self.maze=m
        self.initPlaces=ip
        self.hisDict=hd
        self.initPlacesM=im
        self.hisDictM=hdm
        self.initPlacesQ=q
        self.hisDictQ=hdq
        self.clearList=c
        self.time=-1
        self.displayedTime=-1
        self.mPlaying=False
        
    def display(self):
        t=threading.Thread(target=self.createAndShowGUI)
        t.start()
        #self.createAndShowGUI()
        
    def createAndShowGUI(self):
        self.root=Tk()
        self.root.title("Main Frame")
        content = ttk.Frame(self.root)
        self.mapWidth=500
        self.mapHeight=500
        self.frame = Canvas(content, width=self.mapWidth, height=self.mapHeight)
        self.addMapPanel(self.frame)
        namelbl = ttk.Label(content, text="Name")
        name = ttk.Entry(content)

        back = ttk.Button(content, text="Back")
        back.bind('<Button-1>',self.moveBack)
        play = ttk.Button(content, text="Play")
        #play.bind('<Button-1>',self.runOrPause)
        forward = ttk.Button(content, text="Forward")
        forward.bind('<Button-1>',self.moveForward)
        optimal = ttk.Button(content, text="Optimal")
        optimal.bind('<Button-1>',self.addOptRoutes)
        close = ttk.Button(content, text="Close")
        close.bind('<Button-1>',self.close)

        content.grid(column=0, row=0)
        self.frame.grid(column=0, row=0, columnspan=3, rowspan=2)
        namelbl.grid(column=3, row=0, columnspan=2)
        name.grid(column=3, row=1, columnspan=2)
        back.grid(column=0, row=3)
        play.grid(column=1, row=3)
        forward.grid(column=2, row=3)
        optimal.grid(column=3, row=3)
        close .grid(column=4, row=3)

        self.root.mainloop()

    def addMapPanel(self, canvas):
        self.routesDisplayed=False        
        self.displayMaze(canvas, self.displayedTime, True)
        
    def displayMaze(self,canvas, t0, original=False):
        placement=[]
        if (original):
            placement=self.initPlaces
            placementM=self.initPlacesM
            placementQ=self.initPlacesQ
        else:
            placement=self.hisDict.get(t0)
            placementM=[]
            placementQ=[]
            if(len(self.hisDictM)>0):
                placementM=self.hisDictM.get(t0)
            if(len(self.hisDictQ)>0):
                placementQ=self.hisDictQ.get(t0)
            clear=clearList[t0]
        
        print(placement,placementM,placementQ)    
        xwidth= math.floor((self.mapWidth)/self.maze.getWidth())  
        ywidth= math.floor((self.mapHeight)/self.maze.getHeight())  
        squaresize=min(xwidth,ywidth)
        
        for a in range(0,self.mapWidth+squaresize,squaresize):
            for b in range(0,self.mapHeight+squaresize,squaresize):
                if original:
                    if(self.maze.getValue(int(b/squaresize),int(a/squaresize))==1):
                        canvas.create_rectangle(b,a,b+squaresize,a+squaresize, fill="black")
                    else:
                        canvas.create_rectangle(b,a,b+squaresize,a+squaresize, fill="white")
                else:                    
                    if(Cord(int(b/squaresize),int(a/squaresize)) in clear):
                        canvas.create_rectangle(b,a,b+squaresize,a+squaresize, fill="white")
                    else:
                        canvas.create_rectangle(b,a,b+squaresize,a+squaresize, fill="black")
                    
        if(self.routesDisplayed):
            routes=self.maze.GetOptimalRoute()
            for route in routes:
                for c in range(0,self.mapWidth+squaresize,squaresize):
                    for d in range(0,self.mapHeight+squaresize,squaresize):
                        if(Cord(int(d/squaresize),int(c/squaresize))in route):
                            canvas.create_rectangle(d,c,d+squaresize,c+squaresize, fill="yellow")
          
        #print(placement, placementM)
        for e in range(0,self.mapWidth+squaresize,squaresize):
            for f in range(0,self.mapHeight+squaresize,squaresize):                
                if((int(f/squaresize),int(e/squaresize)) in placement):                    
                    canvas.create_rectangle(f,e,f+squaresize,e+squaresize, fill="blue")
                if((int(f/squaresize),int(e/squaresize)) in placementM):                    
                    canvas.create_rectangle(f,e,f+squaresize,e+squaresize, fill="red")
                if((int(f/squaresize),int(e/squaresize)) in placementQ):                    
                    canvas.create_rectangle(f,e,f+squaresize,e+squaresize, fill="green")
                    
        
        for x in range(0,self.mapWidth+squaresize,squaresize):
            canvas.create_line(x, 0, x, self.maze.getHeight()*squaresize, fill="#476042")
        for y in range(0,self.mapHeight+squaresize,squaresize):
            canvas.create_line(0, y, self.maze.getWidth()*squaresize, y, fill="#476042")
            
    def close(self,event):
        self.root.destroy()
        
    def addOptRoutes(self, event):
        if(self.routesDisplayed):
            self.routesDisplayed=False
        else:
            self.routesDisplayed=True
        self.displayMaze(self.frame, self.displayedTime)
        
    def moveForward(self,event=None):
        if(self.displayedTime==self.time):            
            self.displayedTime=min(self.displayedTime+1,len(self.hisDict)-1)
            self.time=min(self.time+1,len(self.hisDict)-1)
            self.displayMaze(self.frame, self.displayedTime)
            
        elif(self.displayedTime<self.time):
            self.displayedTime=min(self.displayedTime+1,len(self.hisDict)-1)
            self.displayMaze(self.frame, self.displayedTime)
            
    def moveBack(self,event):
        if (self.mPlaying):
            pass#self.runOrPause()
        if(self.displayedTime==0):
            self.displayMaze(self.frame, self.displayedTime)
            return
        self.displayedTime=max(self.displayedTime-1,0)
        self.displayMaze(self.frame, self.displayedTime)
        
    def runOrPause(self,event):
        if(self.mPlaying):      
            self.mPlaying=False
        else:     
            self.mPlaying=True
            while self.mPlaying:
                time.sleep(0.5)
                self.moveForward()
                

Importing Jupyter notebook from /home/esha/anaconda3/Scripts/gym-game/Maze/Maze.ipynb
Importing Jupyter notebook from /home/esha/anaconda3/Scripts/gym-game/Maths/Cord.ipynb


In [2]:
def convert(StringCord):
    intCord=(int(StringCord.split(",")[0].split("(")[1]),int(StringCord.split(",")[1].split(")")[0]))
    return intCord

def convertToClear(clearString, maze):
    clear=[]
    cords=clearString[1:len(clearString)-1].split(")(")
    for cord in cords:
        clear.append(Cord(int(cord.split(",")[0]),int(cord.split(",")[1])))
    clear.append(maze.Initial)
    for exit in maze.Exits:
        clear.append(exit)
    return clear

In [4]:
import nbimporter
import sys
if('..' not in sys.path):
    sys.path.insert(0,'..')
from Windows.MainWindow import MainWindow
from Maze.Maze import Maze


#for book-keeping/need to update with spiders
path= 'DATA/Testing/exp2/Span_5Dim_20_20/'

file=open(path+"/"+"GamesData.txt","r")
f1=file.readlines()
#for line in f1:
    #print(line)    

chosenLine=0
state=f1[chosenLine]
parts=state.split("|")
history=parts[1:(len(parts)-1)]
m=parts[0]

#print(history)
#print("====")

initPlacesP=[]
initPlacesQ=[]
initPlacesM=[]
for i in range(len(history[0].split("#"))-1):
    if "Worker" in str(history[0].split("#")[i+1].split("-")[0]):
        initPlacesP.append(convert(history[0].split("#")[i+1].split("-")[1]))
    elif "Spider" in str(history[0].split("#")[i+1].split("-")[0]):
        initPlacesM.append(convert(history[0].split("#")[i+1].split("-")[1]))
    elif "Queen" in str(history[0].split("#")[i+1].split("-")[0]):
        initPlacesQ.append(convert(history[0].split("#")[i+1].split("-")[1]))
#print(initPlaces)

hisDictPrey={}
hisDictQueen={}
hisDictPredator={}
clearList=[]
maze= Maze(m)
for timeStep in history[1:len(history)]:
    #print(" ")
    #print("Timestep")
    #print(timeStep)
    parts2=timeStep.split("#")
    #print(" Parts2 ")
    #print(parts2)
    clearString=parts2[1]
    clear=convertToClear(clearString,maze)
    clearList.append(clear)
    posP=[]
    posQ=[]
    posM=[]
    for i in range(0,len(parts2)-2):
        #print(parts2[i+2].split("-")[0],parts2[i+2].split("-")[1])
        if "Worker" in str(parts2[i+2].split("-")[0]):
            posP.append(convert(parts2[i+2].split("-")[1]))
        elif "Spider" in str(parts2[i+2].split("-")[0]):
            posM.append(convert(parts2[i+2].split("-")[1]))
        elif "Queen" in str(parts2[i+2].split("-")[0]):
            posQ.append(convert(parts2[i+2].split("-")[1]))
    #print(int(parts2[0]))
    if(len(posP)>0):        
        hisDictPrey[int(parts2[0])]=posP
    if(len(posM)>0):
         hisDictPredator[int(parts2[0])]=posM
    if(len(posQ)>0):
         hisDictQueen[int(parts2[0])]=posQ
#print(hisDict)
#print(history)

#print(initPlaces)
#print(" ")
#print(hisDict)
#print(len(hisDictPrey),len(clearList))
#print(hisDictPredator)
mw=Viewer(maze,initPlacesP,hisDictPrey, initPlacesM, hisDictPredator,initPlacesQ, hisDictQueen, clearList)
mw.display() 

[(5, 0), (5, 0), (5, 0), (5, 0), (5, 0)] [] []
[(6, 0), (6, 0), (5, 0), (5, 0), (5, 0)] [] []
[(6, 0), (7, 0), (5, 0), (5, 0), (5, 0)] [] []
[(6, 0), (7, 0), (5, 0), (5, 0), (5, 0)] [] []
[(6, 0), (7, 0), (5, 0), (5, 0), (5, 0)] [] []
[(6, 0), (7, 0), (5, 0), (5, 0), (5, 0)] [] []
[(6, 0), (7, 0), (5, 0), (6, 0), (5, 0)] [] []
[(6, 0), (7, 0), (5, 0), (6, 0), (5, 0)] [] []
[(6, 0), (7, 0), (5, 0), (7, 0), (6, 0)] [] []
[(6, 0), (7, 0), (5, 0), (7, 0), (6, 0)] [] []
[(6, 0), (7, 0), (5, 0), (8, 0), (7, 0)] [] []
[(6, 0), (8, 0), (6, 0), (8, 0), (7, 0)] [] []
[(7, 0), (9, 0), (6, 1), (8, 0), (7, 0)] [] []
[(8, 0), (9, 0), (6, 1), (8, 0), (8, 0)] [] []
[(8, 0), (9, 0), (6, 1), (9, 0), (8, 0)] [] []
[(8, 0), (9, 0), (6, 1), (9, 0), (9, 0)] [] []
[(8, 0), (9, 0), (6, 1), (9, 0), (9, 0)] [] []
[(8, 0), (9, 0), (6, 1), (9, 0), (9, 0)] [] []
[(8, 0), (9, 0), (6, 1), (9, 0), (9, 0)] [] []
[(9, 0), (9, 0), (6, 1), (9, 0), (9, 0)] [] []
[(9, 0), (9, 0), (6, 1), (9, 0), (9, 0)] [] []
[(9, 0), (9, 

[(9, 0), (9, 0), (6, 2), (9, 0), (9, 0)] [] []
[(9, 0), (9, 0), (6, 2), (9, 0), (9, 0)] [] []
[(9, 0), (9, 0), (6, 2), (9, 0), (9, 0)] [] []
[(9, 0), (9, 0), (6, 2), (9, 0), (9, 0)] [] []
[(9, 0), (9, 0), (6, 2), (9, 0), (9, 0)] [] []
[(9, 0), (9, 0), (6, 2), (9, 0), (9, 0)] [] []
[(9, 0), (9, 0), (6, 3), (9, 0), (9, 0)] [] []
[(9, 0), (9, 0), (7, 3), (9, 0), (9, 0)] [] []
[(9, 0), (9, 0), (7, 3), (9, 0), (9, 0)] [] []
[(9, 0), (9, 0), (7, 3), (9, 0), (9, 0)] [] []
[(9, 0), (9, 0), (7, 3), (9, 0), (9, 0)] [] []
[(9, 0), (9, 0), (7, 3), (9, 0), (9, 0)] [] []
[(9, 0), (9, 0), (7, 3), (9, 0), (9, 0)] [] []
[(9, 0), (9, 0), (7, 3), (9, 0), (9, 0)] [] []
[(9, 0), (9, 0), (7, 3), (9, 0), (9, 0)] [] []
[(9, 0), (9, 0), (7, 3), (9, 0), (9, 0)] [] []
[(9, 0), (9, 0), (7, 3), (9, 0), (9, 0)] [] []
[(9, 0), (9, 0), (7, 3), (9, 0), (9, 0)] [] []
[(9, 0), (9, 0), (7, 3), (9, 0), (9, 0)] [] []
[(9, 0), (9, 0), (7, 3), (9, 0), (9, 0)] [] []
[(9, 0), (9, 0), (7, 3), (9, 0), (9, 0)] [] []
[(9, 0), (9, 